<a href="https://colab.research.google.com/github/kaleem500bc/Simple-NLP-linear-model/blob/master/NLP_simple_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torchtext --upgrade

     |████████████████████████████████| 4.5MB 8.3MB/s 
     |████████████████████████████████| 1.1MB 56.5MB/s 
  Found existing installation: torchtext 0.3.1
    Uninstalling torchtext-0.3.1:
      Successfully uninstalled torchtext-0.3.1


In [2]:
import torch
from torch import nn
import torchvision
from torchvision import transforms
from matplotlib import pyplot as plt
import random
from torch.utils.tensorboard import SummaryWriter
# import torchviz
from torch.nn import functional as F
import torchtext 
from torchtext import datasets as ttDatasets
from torchtext.data.utils import get_tokenizer
%load_ext tensorboard

In [3]:
train_dataset,test_dataset = torchtext.datasets.text_classification.DATASETS['AG_NEWS']('agNews',
                  ngrams=2,vocab=None
                  )

ag_news_csv.tar.gz: 11.8MB [00:00, 42.1MB/s]
120000lines [00:07, 16570.48lines/s]
120000lines [00:16, 7083.41lines/s]
7600lines [00:00, 7711.60lines/s]


In [4]:
def collate_fn(data):
  labels = torch.tensor([l for l,t in data])
  text = [t for l,t in data]
  textLen = [0]+[len(t) for l,t in data]
  offsets = torch.tensor(textLen[:-1]).cumsum(dim=0)
  text = torch.cat(text)
  return text,labels,offsets

In [5]:
BatchSize= 16
dataloader = torch.utils.data.DataLoader(train_dataset,BatchSize,shuffle=True,collate_fn=collate_fn)


In [6]:
class lModel(nn.Module):
  def __init__(self,vocab_size):
    super().__init__()
    self.embd = nn.EmbeddingBag(vocab_size,2)
    self.f1 = nn.Linear(2,32)
    self.f2 = nn.Linear(32,64)
    self.f3 = nn.Linear(64,32)
    self.f = nn.Linear(32,4)
  def forward(self,text,offsets):
    x = self.embd(text,offsets)
    x = F.relu(self.f1(x))
    x = F.relu(self.f2(x))
    x = F.relu(self.f3(x))
    x = F.relu(self.f(x))
    return x

In [7]:
vocab_size = len(train_dataset.get_vocab())
linearModel = lModel(vocab_size)

In [8]:
criterion = nn.CrossEntropyLoss()
optim = torch.optim.Adam(linearModel.parameters())

linearModel.to("cuda")

lModel(
  (embd): EmbeddingBag(1308844, 2, mode=mean)
  (f1): Linear(in_features=2, out_features=32, bias=True)
  (f2): Linear(in_features=32, out_features=64, bias=True)
  (f3): Linear(in_features=64, out_features=32, bias=True)
  (f): Linear(in_features=32, out_features=4, bias=True)
)

In [9]:
epoch = 5
for e in range(epoch):
  totalLoss = 0
  for txt,lbl,offset in dataloader:
    txt, offset, lbl = txt.to("cuda"),offset.to("cuda"), lbl.to("cuda")

    out = linearModel(txt,offset)
    optim.zero_grad()    
    loss = criterion(out,lbl)
    # print(loss)
    loss.backward()
    totalLoss = totalLoss + loss.item()
    optim.step()
  print(totalLoss)

6250.1799277588725
3403.237520221621
2536.8534248247743
2022.162873136811
1642.9876332059503


In [10]:
class_name = {0:'world',1:'sports',2:'Business',3:'sci/tech'}


sentence = "MEMPHIS, Tenn. – Four days ago, Jon Rahm was \
    enduring the season’s worst weather conditions on Sunday at The \
    Open on his way to a closing 75 at Royal Portrush, which \
    considering the wind and the rain was a respectable showing. \
    Thursday’s first round at the WGC-FedEx St. Jude Invitational \
    was another story. With temperatures in the mid-80s and hardly any \
    wind, the Spaniard was 13 strokes better in a flawless round. \
    Thanks to his best putting performance on the PGA Tour, Rahm \
    finished with an 8-under 62 for a three-stroke lead, which \
    was even more impressive considering he’d never played the \
    front nine at TPC Southwind."

tokenizer = torchtext.data.utils.get_tokenizer('basic_english')

vocab = train_dataset.get_vocab()
tokens = tokenizer(sentence)
ngrams = torchtext.data.utils.ngrams_iterator(tokens,2)
text = torch.tensor([vocab[i] for i in ngrams]).to("cuda")


out = linearModel(text,torch.tensor([0]).to("cuda"))
print(class_name[torch.argmax(out).item()])




sports
